# Schema
* Header
  * Address
    * Street
    * City
    * State
    * Zip
  * Phone
* Body
  * DateTime
  * REF
  * AID
  * TC
  * RewardsID
  * Tax
  * Balance
  * Paid
  * Change
  * TotalSavings
  * TotalCoupons
  * Items (Array)
    * Product
    * Quantity (optional)
    * Price
    * Savings
  
* Footer

In [1]:
import pandas as pd
import re

In [2]:
from decimal import *

In [3]:
basepath = '/Users/ericmelz/Desktop/Ralphs/Receipts/Excel'
receipt1 = f'{basepath}/2020_02_10_01.xlsx'

In [4]:
df = pd.read_excel(receipt1)

```
general strategies
have high level functions for extraction

extractHeader calls
  extractAddress calls
    extractFirstLine - extracts first line of receipt - which is always street address
  extractFirstPhoneNumber - finds first occurrence of (ddd) ddd-dddd pattern
  extractCashier - finds "Your cashier was <blah>"
  extractCityStateZip - find first occurence of <city> <XX> <ddddd>

  returns header object, with metadata about rows used
    
    
extractBody
  locate dateTime
  extract everything through dateTime
  eliminate rows that were included in Header
 ```

In [5]:
def find_and_extract(pattern, df):
    '''
    Find a pattern in df and extract the matching group.
    Return the extracting and the (row, col) location. 
    If not found or not matched, return None
    '''
    for col in range(df.shape[1]):
        df2 = df[df.columns[col]].str.findall(pattern).str.len() > 0
        if not df2.any():
            continue
        row = df[df[df.columns[col]].str.findall(pattern).str.len() > 0].index[0]
        contents = df.iloc[row, col]
        m = re.match(pattern, contents)
        if m.lastindex < 1:
            return None
        extracted = []
        for i in range(1, m.lastindex + 1):
            extracted.append(m[i])
        return extracted, (row, col)
    return None

In [6]:
class Address:
    street = None
    city = None
    state = None
    zipcode = None
    
    def __init__(self, street, city, state, zipcode):
        self.street = street
        self.city = city
        self.state = state
        self.zipcode = zipcode
    
    def __str__(self):
        return f'{self.street} {self.city} {self.state} {self.zipcode}'

In [7]:
def extract_address(df):
    '''
    Return an address object
    '''
    city_state_zip_pattern = '(.*) (..) (\d{5})'
    street_pattern = '(\d+ .* (Blvd|St))'
    street, rowcol = find_and_extract(street_pattern, df)
    city_state_zip, rowcol = find_and_extract(city_state_zip_pattern, df)
    city = city_state_zip[0]
    state = city_state_zip[1]
    zipcode = city_state_zip[2]
    address = Address(street[0], city, state, zipcode)
    return address
    
    

In [8]:
def extract_phone(df):
    phone, rowcol = find_and_extract('(\(\d{3}\) \d{3}-\d{4})', df)
    return phone[0]

In [9]:
def extract_cashier(df):
    cashier, rowcol = find_and_extract('Your cashier was (.*)', df)
    return cashier[0], rowcol

In [10]:
class Header:
    address = None
    cashier = None
    phone = None
    
    def __init__(self, address, cashier, phone):
        self.address = address
        self.cashier = cashier
        self.phone = phone
        
    def __str__(self):
        return f'[address: {self.address}, cashier: {self.cashier}, phone: {self.phone}]'

In [11]:
def extract_header(df):
    address = extract_address(df)
    cashier, _ = extract_cashier(df)
    phone = extract_phone(df)
    header = Header(address, cashier, phone)
    return header

```
Line items strategy
segment line items
  - first non-blank line after cashier
  - RALPHS reward CUSTOMER
  
tokenize non-blank cells into stream
- item @ quantity applies to following item
- RALPHS SAVED YOU + quantity applies to preceding item unless it's CA REDEM VAL then it's 2 items back (the bottle)
- Can Ignore SC, RC (noise)
- When we see amounts, strip off F or B. 
  - If previous token is a RALPHS_SAVED_YOU, apply the savings to the previous token (or 2 back)
  - otherwise, apply the total to the product
  
Example:
['8 @ 1.29', 'CGYS ALPINE', 'RC', '10.00 F', '8 @ .10', 'CA REDM VAL', '0.80 F', 
 'RALPHS SAVED YOU', '0.32', 'ARROWHEAD ICE', '4.49 B']
 
=>

[ItemsAndQuantity('8', '1.29'), Product('CGYS ALPINE'), ItemTotal('10.00'), 
 ItemsAndQuantity('8', '0.10'), Product('CA REDEM VAL'), ItemTotal('0.80'),
 Savings('0.32'),
 Product('ARROWHEAD ICE'), ItemTotal('4.49')]
 
=>
[LineItem(product='CGYS ALPINE', qty=8, unitprice=1.29, total=10.00, savings=.32)
 LineItem(product='CA REDEM VAL', qty=8, unitprice=0.10, total=0.80)
 LineItem(product='ARROWHEAD ICE', total=4.49)] 

tokenization rules:

- match (\d+) @ (\d*\.\d\d)
  emit ItemQuantity({1}, {2})
  
- match SC, RC - skip

- match (\d+\.\d\d)
  emit ItemTotal

- match RALPHS SAVED YOU
  - lookahead to next token
    - confirm match of (\d+\.\d{2})
    - emit Savings({1})

- otherwise match (.*)
  emit Product({1})  

```

In [12]:
class ItemsAndQuantity:
    qty = None
    unitprice = None
    
    def __init__(self, qty, unitprice):
        self.qty = int(qty)
        self.unitprice = Decimal(unitprice)
    
    def __str__(self):
        return f'ItemsAndQuantity({self.qty}, {self.unitprice})'

In [13]:
class Product:
    name = None
    
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f'Product({self.name})'

In [14]:
class ItemTotal:
    amount = None
    
    def __init__(self, amount):
        self.amount = Decimal(amount)
    
    def __str__(self):
        return f'ItemTotal({self.amount})'

In [15]:
class Savings:
    amount = None
    
    def __init__(self, amount):
        self.amount = Decimal(amount)
    
    def __str__(self):
        return f'Savings({self.amount})'

In [16]:
class LineItem:
    product = None
    qty = None
    unitprice = None
    total = None
    savings = None
    
    def __str__(self):
        return f'LineItem(product={self.product}, qty={self.qty}, unitprice={self.unitprice}, total={self.total}, ' + \
            f'savings={self.savings})'

In [17]:
class Noise:
    noise = None
    
    def __init__(self, noise):
        self.noise = noise
        
    def __str__(self):
        return f'Noise({self.noise})'

In [18]:
def parse_items_and_quantity(text):
    m = re.match('(\d) @ (\d*\.\d{2})', text)
    if m:
        return ItemsAndQuantity(m[1], m[2])
    return None

In [19]:
def parse_item_total(text):
    m = re.match('(\d+\.\d{2})', text)
    if m:
        return ItemTotal(m[1])
    return None

In [20]:
def parse_savings_header(text):
    m = re.match('RALPHS SAVED YOU', text)
    if m:
        return True
    return None

In [21]:
def parse_savings_body(text):
    m = re.match('(\d+\.\d{2})', text)
    if m:
        return Savings(m[1])
    return None

In [22]:
def parse_product(text):
    return Product(text)

In [23]:
def parse_noise(text):
    m = re.match('(SC|RC)', text)
    if m:
        return Noise(m[1])
    return None

In [24]:
def parse_text(texts):
    '''
    Given a list of texts (strings) produce a list of tokens
    '''
    tokens = []
    i = -1
    while i < len(texts) - 1:
        i += 1
        text = texts[i]
        token = parse_items_and_quantity(text)        
        if token:
            tokens.append(token)
            continue
        token = parse_noise(text)
        if token:
            continue
        token = parse_savings_header(text)
        if token:
            i += 1
            text = texts[i]
            token = parse_savings_body(text)
            if token:
                tokens.append(token)
                continue
        token = parse_item_total(text)
        if token:
            tokens.append(token)
            continue
        token = parse_product(text)
        tokens.append(token)
    return tokens
        

In [25]:
def parse_tokens(tokens):
    '''
    Given a list of tokens produce a list of line items
    '''
    items = []
    i = -1
    while i < len(tokens) - 1:
        i += 1
        token = tokens[i]
        if isinstance(token, ItemsAndQuantity):
            item = LineItem()
            item.qty = token.qty
            item.unitprice = token.unitprice
            items.append(item)
            continue
        if isinstance(token, Product):
            if item.total is not None:
                item = LineItem()
                items.append(item)
            item.product = token.name
            continue
        if isinstance(token, ItemTotal):
            item.total = token.amount
            continue
        if isinstance(token, Savings):
            tempitem = item
            if item.product == 'CA REDEM VAL':
                tempitem = items[-2]
            tempitem.savings = token.amount
    return items
        

In [26]:
def find_rewards_customer_line(df):
    line, rowcol = find_and_extract('(RALPHS rewards CUSTOMER)', df)
    return rowcol[0]

In [27]:
def extract_line_item_text(df, start, end):
    texts = []
    for r in range(start, end):
        for c in range(df.shape[1]):
            text = df.iloc[r][c]
            if text and str(text) != 'nan':
                texts.append(str(text))
    return texts

In [28]:
def extract_line_items(df):
    _, (row, _) = extract_cashier(df)
    lineitem_start = row + 1
    lineitem_end = find_rewards_customer_line(df)
    texts = extract_line_item_text(df, lineitem_start, lineitem_end)
    tokens = parse_text(texts)
    items = parse_tokens(tokens)
    return items

In [29]:
class Body:
    line_items = None
    
    def __init__(self, line_items):
        self.line_items = line_items
        
    def __str__(self):
        return '\n'.join([str(x) for x in self.line_items])

In [30]:
def extract_body(df):
    line_items = extract_line_items(df)
    body = Body(line_items)
    return body

In [31]:
class Receipt:
    header = None
    Body = None
    
    def __init__(self, header, body):
        self.header = header
        self.body = body
        
    def __str__(self):
        return str(self.header) + '\n' + str(self.body)

In [32]:
def extract_receipt(df):
    header = extract_header(df)
    body = extract_body(df)
    receipt = Receipt(header, body)
    return receipt

In [33]:
receipt = extract_receipt(df)

In [34]:
print(receipt)

[address: 9616 W. Pico Blvd Los Angeles CA 90035, cashier: CHEC 504, phone: (310) 271-2672]
LineItem(product=CGYS ALPINE, qty=8, unitprice=1.29, total=10.00, savings=0.32)
LineItem(product=CA REDEM VAL, qty=8, unitprice=0.10, total=0.80, savings=None)
LineItem(product=ARROWHEAD ICE, qty=None, unitprice=None, total=4.49, savings=None)


## TODO
* Save receipts as json to disk
* Try more receipts
* Compute total
* Handle Tax
* Extract footer
* Extract datetime
* Save date and sequence number (from filename) as fields
* Cleanup, reorg logic
* Save as script
* Run from cmd line
* Test - compare calculated totals with reported totals